# Transform Rasters
- This notebook demonstrates how to calculate the residual topography transformation to an extent DEM using ScrollStats.
- Extent DEMs are the DEMS roughly clipped to the extent, not the exact boundary, of the ridge and swale topography of a bend. 
- The profile curvature transformation is calculated using QGIS. Instructions can be found at the end of this notebook
- It is reccommended that extent DEMs be saved in the `data/input/dems` folder 
- The calulated residual topography transform of the dems will be saved to the `data/output/rt` directory
- Once both transforms (residual topography and profile curvature) are completed, open [ClassifyRasters.ipynb](ClassifyRasters.ipynb) to complete the ridge area classification




In [ ]:
from pathlib import Path

import numpy as np
import rasterio
import matplotlib.pyplot as plt

from scrollstats.delineation import CalcResidualTopography
from parameters import RASTER_WINDOW_SIZE

# Define Inputs


In [ ]:
# Define processing parameters
window_size = RASTER_WINDOW_SIZE

# Define input/output file paths
dem_path = Path("example_data/input/LBR_025_dem.tif")
output_dir = Path("example_data/output")


## Calculate Residual Topography
- provide the dem path, the processing window size, and the output directory to the `CalcResidualTopography` class
- Call the `.execute()` method to calculate the transformation and save the transformed raster to the output directory
- This process may take several seconds to complete; especially for larger window sizes. 
    - ~16s for `LBR_025_dem.tif` with a 45px window size

In [ ]:
# Instantiate the class
rt = CalcResidualTopography(dem_path, window_size, output_dir)
rt_path = rt.execute()
print(f"RT raster saved to {rt_path}")

## DEM vs Residual Topography example 

In [ ]:
dem = rasterio.open(dem_path).read(1)
dem[dem<-1e6] = np.nan

rt = rasterio.open(rt_path).read(1)

fig, (ax1,ax2) = plt.subplots(1, 2, figsize=(20, 15))
mapper = ax1.imshow(dem)
# fig.colorbar(mapper, ax=ax1)
ax1.set_axis_off()
ax1.set_title("Original DEM")


mapper = ax2.imshow(rt)
fig.colorbar(mapper, ax=ax2)
ax2.set_axis_off()
ax2.set_title("Residual Topography");


## Instructions for QGIS implementation of profile curvature

Profile curvature is calculated using the `r.param.scale` tool in QGIS3

The script `profileCurvature_QGIS.py` (located at [scrollstats/delineation/profileCurvature_QGIS.py](scrollstats/delineation/profileCurvature_QGIS.py)) uses the `r.param.scale` tool and should be used to calculate the profile curvature transform for the DEM. 

### Using `profileCurvature_QGIS.py`
Execute `profileCurvature_QGIS.py` in the python console of QGIS. 

You can open the python console by clicking the python symbol in the tool bar
![launch python console](img/launch_python_console.png)

Then click the "show editor" button and open the script.
![show editor](img/open_script_editor.png)

Alter the input parameters and run the script
- set the `dem_path`, `output_dir`, and `window_size` variables to the same values used in this notebook.
![run the script](img/run_script.png)

